In [1]:
import gradio as gr
import pandas as pd
import joblib
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from utils import set_label_encoding
from datetime import datetime
from tabulate import tabulate

In [2]:
# labels - column to use in resulting from model and used in gradio
target_labels = ['Amoxicilline', 'Augmentin', 'Oxacilline / cefazoline', 'Tazocilline', 'Cefotaxime / ceftriaxone', 'Ceftazidime', 'Cefepime', 'Aztreonam', 'Imipenem', 'Meropenem', 'Ertapenem', 'Amikacine', 'Gentamicine', 'Ciprofloxacine', 'Levofloxacine', 'Bactrim', 'Vancomycine','Rifampicine','Clindamycine''Macrolides']

In [3]:
# df = pd.read_excel('Calmette_data_v2.xlsx')
df = pd.read_excel('antibiogram_dataset.xlsx')

# remove '\u200b'
df = df.replace(to_replace='\u200b', value='', regex=True)

# drop columns
df = df.drop(columns=["Hopital", "IDLabo", "IDPatient", "visittype"])
df = df.rename(columns={"new_age": "age"})

# Convert column names to lowercase
df.columns = df.columns.str.lower()

# Mapping Khmer to English using map()
df['sex'] = df['sex'].map({'ប្រុស': 'Male', 'ស្រី': 'Female'})

In [4]:
# df['new_diagnosis'].unique().tolist()

In [5]:
# Set Label Encoding to convert features to number
sex_dict = set_label_encoding(df, 'sex')
address_dict = set_label_encoding(df, 'address')
ward_dict = set_label_encoding(df, 'ward_en')
service_type_dict = set_label_encoding(df, 'service_type')
sample_dict = set_label_encoding(df, 'prelevement_type')
germe_dict = set_label_encoding(df, 'germe')
espece_requete_dict = set_label_encoding(df, 'espece_requete')
direct_2_dict = set_label_encoding(df, '2_direct')
culture_3_dict = set_label_encoding(df, '3_culture')
genre_4_dict = set_label_encoding(df, '4_genre')
espece_5_training_dict = set_label_encoding(df, '5_espece_training')

diagnosis_dict = set_label_encoding(df, 'new_diagnosis') ## add diagnosis

In [6]:
def amr_project(age, sex, address, ward_en, service_type, date, diagnosis, sample, direct_2, culture_3, genre_4, species_5):

    # Convert timestamp to a datetime object
    date_time = datetime.fromtimestamp(date)
    # Extract the month
    month = date_time.month

    # Stage 5
    if age and sex and address and ward_en and service_type and sample and direct_2 and culture_3 and genre_4 and species_5:
        print('Stage 5')

        # Sample data
        input_data = {
            "month": [month],
            "age": [age],
            'sex': [sex_dict[sex]],
            'address': [address_dict[address]],
            'ward_en': [ward_dict[ward_en]],
            'service_type': [service_type_dict[service_type]],
            'sample': [sample_dict[sample]],
            'new_diagnosis': [diagnosis_dict[diagnosis]],
            '2_direct': [direct_2_dict[direct_2]],
            '3_culture': [culture_3_dict[culture_3]],
            '4_genre': [genre_4_dict[genre_4]],
            '5_species': [espece_5_training_dict[species_5]]
        }
        # Create the DataFrame
        df = pd.DataFrame(input_data)
        print(tabulate(df, headers="keys", tablefmt="fancy_grid"))
        # print(df)

        # Load the model back from the file
        rf_model_loaded = joblib.load('random_forest_stage_5.joblib')
        one_row_test = df

        # Now you can use the loaded model to make predictions
        y_pred_loaded = rf_model_loaded.predict(one_row_test)
        # print(len(y_pred_loaded[0]), np.array(y_pred_loaded))

        y_pred_prob = rf_model_loaded.predict_proba(one_row_test)
        # print(len(y_pred_prob), np.array(y_pred_prob))

        result_class_dict = dict()
        result_probab_dict = dict()

        for i in range(len(target_labels)):
            message = "Sensible" if y_pred_loaded[0][i] == 1 else "Resistance"
            print(target_labels[i], ": class - ", y_pred_loaded[0][i], message,' - Probab:', y_pred_prob[i][0][1])
            result_class_dict[target_labels[i]] = message
            result_probab_dict[target_labels[i]] = y_pred_prob[i][0][1]

        print()
        print(result_class_dict)
        print(result_probab_dict)
        
        # Sample data
        data = {
            "Model": ["Random Forest - Stage 5"]
        }
        # Create the DataFrame
        df_info = pd.DataFrame(data)
        print(tabulate(df_info, headers="keys", tablefmt="fancy_grid"))


    # Stage 4
    elif age and sex and address and ward_en and service_type and sample and direct_2 and culture_3 and genre_4:
        print('Stage 4')
        # Sample data
        input_data = {
            "month": [month],
            "age": [age],
            'sex': [sex_dict[sex]],
            'address': [address_dict[address]],
            'ward_en': [ward_dict[ward_en]],
            'service_type': [service_type_dict[service_type]],
            'sample': [sample_dict[sample]],
            'new_diagnosis': [diagnosis_dict[diagnosis]],
            '2_direct': [direct_2_dict[direct_2]],
            '3_culture': [culture_3_dict[culture_3]],
            '4_genre': [genre_4_dict[genre_4]]
        }
        # Create the DataFrame
        df = pd.DataFrame(input_data)
        print(tabulate(df, headers="keys", tablefmt="fancy_grid"))
        # print(df)
        # Load the model back from the file
        rf_model_loaded = joblib.load('random_forest_stage_4.joblib')
        one_row_test = df

        # Now you can use the loaded model to make predictions
        y_pred_loaded = rf_model_loaded.predict(one_row_test)
        # print(len(y_pred_loaded[0]), np.array(y_pred_loaded))

        y_pred_prob = rf_model_loaded.predict_proba(one_row_test)
        # print(len(y_pred_prob), np.array(y_pred_prob))

        result_class_dict = dict()
        result_probab_dict = dict()

        for i in range(len(target_labels)):
            message = "Sensible" if y_pred_loaded[0][i] == 1 else "Resistance"
            print(target_labels[i], ": class - ", y_pred_loaded[0][i], message,' - Probab:', y_pred_prob[i][0][1])
            result_class_dict[target_labels[i]] = message
            result_probab_dict[target_labels[i]] = y_pred_prob[i][0][1]

        print()
        print(result_class_dict)
        print(result_probab_dict)

         # Sample data
        data = {
            "Model": ["Random Forest - Stage 4"]
        }
        # Create the DataFrame
        df_info = pd.DataFrame(data)
        print(tabulate(df_info, headers="keys", tablefmt="fancy_grid"))

    # Stage 3
    elif age and sex and address and ward_en and service_type and sample and direct_2 and culture_3:
        print('Stage 3')
        # Sample data
        input_data = {
            "month": [month],
            "age": [age],
            'sex': [sex_dict[sex]],
            'address': [address_dict[address]],
            'ward_en': [ward_dict[ward_en]],
            'service_type': [service_type_dict[service_type]],
            'sample': [sample_dict[sample]],
            'new_diagnosis': [diagnosis_dict[diagnosis]],
            '2_direct': [direct_2_dict[direct_2]],
            '3_culture': [culture_3_dict[culture_3]]
        }
        # Create the DataFrame
        df = pd.DataFrame(input_data)
        print(tabulate(df, headers="keys", tablefmt="fancy_grid"))
        # print(df)
        # Load the model back from the file
        rf_model_loaded = joblib.load('random_forest_stage_3.joblib')
        one_row_test = df

        # Now you can use the loaded model to make predictions
        y_pred_loaded = rf_model_loaded.predict(one_row_test)
        # print(len(y_pred_loaded[0]), np.array(y_pred_loaded))

        y_pred_prob = rf_model_loaded.predict_proba(one_row_test)
        # print(len(y_pred_prob), np.array(y_pred_prob))

        result_class_dict = dict()
        result_probab_dict = dict()

        for i in range(len(target_labels)):
            message = "Sensible" if y_pred_loaded[0][i] == 1 else "Resistance"
            print(target_labels[i], ": class - ", y_pred_loaded[0][i], message,' - Probab:', y_pred_prob[i][0][1])
            result_class_dict[target_labels[i]] = message
            result_probab_dict[target_labels[i]] = y_pred_prob[i][0][1]

        print()
        print(result_class_dict)
        print(result_probab_dict)

         # Sample data
        data = {
            "Model": ["Random Forest - Stage 3"]
        }
        # Create the DataFrame
        df_info = pd.DataFrame(data)
        print(tabulate(df_info, headers="keys", tablefmt="fancy_grid"))

    # Stage 2
    elif age and sex and address and ward_en and service_type and sample and direct_2:
        print('Stage 2')
        # Sample data
        input_data = {
            "month": [month],
            "age": [age],
            'sex': [sex_dict[sex]],
            'address': [address_dict[address]],
            'ward_en': [ward_dict[ward_en]],
            'service_type': [service_type_dict[service_type]],
            'sample': [sample_dict[sample]],
            'new_diagnosis': [diagnosis_dict[diagnosis]],
            '2_direct': [direct_2_dict[direct_2]]
        }
        # Create the DataFrame
        df = pd.DataFrame(input_data)
        print(tabulate(df, headers="keys", tablefmt="fancy_grid"))
        # print(df)
        # Load the model back from the file
        rf_model_loaded = joblib.load('random_forest_stage_2.joblib')
        one_row_test = df

        # Now you can use the loaded model to make predictions
        y_pred_loaded = rf_model_loaded.predict(one_row_test)
        # print(len(y_pred_loaded[0]), np.array(y_pred_loaded))

        y_pred_prob = rf_model_loaded.predict_proba(one_row_test)
        # print(len(y_pred_prob), np.array(y_pred_prob))

        result_class_dict = dict()
        result_probab_dict = dict()

        for i in range(len(target_labels)):
            message = "Sensible" if y_pred_loaded[0][i] == 1 else "Resistance"
            print(target_labels[i], ": class - ", y_pred_loaded[0][i], message,' - Probab:', y_pred_prob[i][0][1])
            result_class_dict[target_labels[i]] = message
            result_probab_dict[target_labels[i]] = y_pred_prob[i][0][1]

        print()
        print(result_class_dict)
        print(result_probab_dict)

         # Sample data
        data = {
            "Model": ["Random Forest -  Stage 2"]
        }
        # Create the DataFrame
        df_info = pd.DataFrame(data)
        print(tabulate(df_info, headers="keys", tablefmt="fancy_grid"))

    # Stage 1
    elif age and sex and address and ward_en and service_type and sample:
        print('Stage 1')
        # Sample data
        input_data = {
            "month": [month],
            "age": [age],
            'sex': [sex_dict[sex]],
            'address': [address_dict[address]],
            'ward_en': [ward_dict[ward_en]],
            'service_type': [service_type_dict[service_type]],
            'sample': [sample_dict[sample]],
            'new_diagnosis': [diagnosis_dict[diagnosis]],
        }
        # Create the DataFrame
        df = pd.DataFrame(input_data)
        print(tabulate(df, headers="keys", tablefmt="fancy_grid"))
        
        # Load the model back from the file
        rf_model_loaded = joblib.load('random_forest_stage_1.joblib')
        one_row_test = df

        # Now you can use the loaded model to make predictions
        y_pred_loaded = rf_model_loaded.predict(one_row_test)
        # print(len(y_pred_loaded[0]), np.array(y_pred_loaded))

        y_pred_prob = rf_model_loaded.predict_proba(one_row_test)
        # print(len(y_pred_prob), np.array(y_pred_prob))

        result_class_dict = dict()
        result_probab_dict = dict()

        for i in range(len(target_labels)):
            message = "Sensible" if y_pred_loaded[0][i] == 1 else "Resistance"
            print(target_labels[i], ": class - ", y_pred_loaded[0][i], message,' - Probab:', y_pred_prob[i][0][1])
            result_class_dict[target_labels[i]] = message
            result_probab_dict[target_labels[i]] = y_pred_prob[i][0][1]

        print()
        print(result_class_dict)
        print(result_probab_dict)

         # Model used and stage information
        data = {
            "Model": ["Random Forest - Stage 1"]
        }
        # Create the DataFrame
        df_info = pd.DataFrame(data)
        print(tabulate(df_info, headers="keys", tablefmt="fancy_grid"))

    return df_info, result_probab_dict
    # return df_info, {'Amoxicilline': 0.74, 'Augmentin': 0.22, 'Oxacilline / cefazoline': 0.62}

app = gr.Interface(fn=amr_project, inputs=[
    gr.Textbox(label="Age"),
    gr.Radio(["Male", "Female"], label="Sex"),
    gr.Dropdown(['Kandal', 'Kampong Cham', 'Takeo', 'Phnom Penh', 'Svay Rieng', 'Kampot', 'Battambang', 'Kampong Thom', 'Kampong Speu', 'Koh Kong', 'Prey Veng', 'Preah Sihanouk', 'Kampong Chhnang', 'Tbong Khmum', 'Pursat', 'Mondulkiri', 'Kratie', 'Banteay Meanchey', 'Preah Vihear', 'Oddar Meanchey', 'Ratanakiri', 'Siem Reap', 'Kep', 'Stung Treng', 'Pailin', 'Unknown'], label="Address"),
    gr.Dropdown(['Maternity', 'Surgery B', 'Medicine B', 'Medicine A4', 'Cancer/blood disorders', 'Emergency', 'Surgery A', 'Medicine A', 'Medicine A5', 'Gyn', 'Neuro-surgery', 'Neurology', 'Surgery A2', 'Cardio Pediatric', 'Cardio-Rea', 'Medicine A6', 'Cardio B', 'Rea-neuro', 'Hepatogastroenterology', 'Rea B', 'Medicine A7', 'Cardio C', 'Cardio A', 'Rea A', 'Rea C'], label="Ward"),
    gr.Dropdown(['Pediatrie', 'Chirurgie', 'Medecine', 'SAU', 'Gyneco/Obs', 'Rea'], label="Service Type"),
    gr.DateTime(label='Date'),
    gr.Dropdown(['Les infections maternelles ', 'Infection du tissu mou et ostéo-articulaire ', 'Infection digestive ', 'Infection and Trouble metabolique(Diabetes…)', 'Infection Respiratoire', 'Complications infectieuses des Accidents vasculaire cérébrales', 'Infection system nerveux central ', 'Choc Septique ', 'Other infection', 'Infection en onco-hématologie ', 'Infection urinaire ', 'Infections genitales', 'Fracture osseuse', 'Infection and HTA, Coeur , Vasculaire ', 'ORL , EYES and ENT', 'Melioidosis', 'Unknown'], label='Diagnosis'),
    gr.Dropdown(['Collection/Abces', 'Hemoc/KT', 'Urine', 'Genital', 'Respiratoire', 'LCR', 'Sereuse', 'Osteoarticulaire', 'Biopsie', 'Digestif', 'Peau', 'Materiel'], label='Sample'),
    gr.Dropdown(['Cocci Gram Pos', 'Bacille Gram Neg', 'Levure', 'Bacille Gram Pos'], label='Stage 2 - Direct'),
    gr.Dropdown(['Cocci Gram Pos type streptocoque', 'Cocci Gram Pos type staphylocoque', 'Bacille Gram Neg type enterobacterie', 'Bacille Gram Neg non fermentant', 'Cocci Gram Pos type enterocoque', 'Levure', 'Cocci Gram Pos non specifie', 'Bacille Gram Neg non specifie', 'Bacille Gram Pos'], label='Stage 3 - Culture'),
    gr.Dropdown(['Streptococcus', 'Staphylococcus', 'Enterobacter', 'Pantoea', 'Burkholderia', 'Acinetobacter', 'Stenotrophomonas', 'Pseudomonas', 'Enterococcus', 'Klebsiella', 'Proteus', 'Candida', 'Lactococcus', 'Salmonella', 'Citrobacter', 'Providencia', 'Morganella', 'Aeromonas', 'Aerococcus', 'Serratia', 'Edwardsiella', 'Plesiomonas', 'Listeria', 'Gemella', 'Shigella', 'Cronobacter'], label='Stage 4 - Genre'),
    gr.Dropdown(['Streptococcus anginosus/constellatus/intermedius/milleri', 'Staphylococcus aureus', 'Enterobacter cloacae complex', 'Pantoea agglomerans', 'Burkholderia cepacia complex', 'Acinetobacter baumannii-calcoaceticus complex', 'Acinetobacter sp.', 'Stenotrophomonas maltophilia', 'Pseudomonas aeruginosa', 'Streptococcus uberis', 'Staphylococcus hominis', 'Staphylococcus epidermidis', 'Enterococcus faecalis', 'Cronobacter sakazakii', 'Staphylococcus saprophyticus', 'Enterococcus faecium', 'Klebsiella pneumoniae', 'Proteus mirabilis', 'Klebsiella ornithinolytica', 'Candida albicans', 'Streptococcus agalactiae/B', 'Streptococcus mitis/australis/cristatus/infantis/massiliensis/oligofermentans/oralis/peroris/pseudopneumoniae/sinensis', 'Streptococcus acidominimus', 'Streptococcus sanguinis/parasanguinis/gordonii', 'Streptococcus equinus(bovis)/gallolyticus(caprinus)/infantarius/lutetiensis/alactolyticus/pasteurianus/D(hors enterococoques)', 'Lactococcus garvieae', 'Streptococcus dysgalactiae/equi/equisimilis/C/canis/G', 'Streptococcus sp.', 'Staphylococcus haemolyticus', 'Salmonella Typhi', 'Salmonella Paratyphi', 'Salmonella sp.', 'Salmonella enterica', 'Enterococcus casseliflavus/gallinarum', 'Enterococcus avium', 'Citrobacter koseri', 'Klebsiella aerogenes', 'Citrobacter freundii complex', 'Klebsiella ozaenae', 'Proteus vulgaris', 'Klebsiella oxytoca', 'Providencia stuartii', 'Morganella morganii', 'Enterococcus durans', 'Burkholderia pseudomallei', 'Streptococcus suis', 'Streptococcus pyogenes/A', 'Aeromonas caviae/dhakensis/eucrenophila/hydrophila/punctata/sobria/veronii', 'Acinetobacter lwoffii/haemolyticus', 'Staphylococcus lugdunensis', 'Staphylococcus caprae', 'Streptococcus pneumoniae', 'Enterococcus gallinarum', 'Enterococcus hirae', 'Enterococcus sp.', 'Candida glabrata', 'Candida tropicalis', 'Candida melibiosica', 'Aerococcus viridans', 'Pseudomonas putida', 'Enterococcus raffinosus', 'Serratia liquefaciens', 'Edwardsiella tarda', 'Plesiomonas shigelloides', 'Listeria monocytogenes', 'Candida sp.', 'Candida parapsilosis', 'Gemella morbillorum', 'Streptococcus salivarius/thermophilus/vestibularis', 'Streptococcus porcinus', 'Staphylocoque a coagulase negative', 'Citrobacter amalonaticus group', 'Klebsiella terrigena', 'Serratia marcescens', 'Providencia rettgeri', 'Shigella sonnei', 'Cronobacter malonaticus'], label='Stage 5 - Species')
    # gr.Dropdown(['Amoxicilline', 'Augmentin', 'Oxacilline / cefazoline', 'Tazocilline', 'Cefotaxime / ceftriaxone', 'Ceftazidime', 'Cefepime', 'Aztreonam', 'Imipenem', 'Meropenem', 'Ertapenem', 'Amikacine', 'Gentamicine', 'Ciprofloxacine', 'Levofloxacine', 'Bactrim', 'Vancomycine', 'Rifampicine', 'Clindamycine', 'Macrolides'], label='Antibiotics')
    ], 
    outputs=[
        gr.Dataframe(label='Information'),
        gr.Label(label='Antibiotics - Sensibility Prediction')
        ]
    )

app.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/Users/thearsophal/opt/miniconda3/lib/python3.9/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Stage 5
╒════╤═════════╤═══════╤═══════╤═══════════╤═══════════╤════════════════╤══════════╤═════════════════╤════════════╤═════════════╤═══════════╤═════════════╕
│    │   month │   age │   sex │   address │   ward_en │   service_type │   sample │   new_diagnosis │   2_direct │   3_culture │   4_genre │   5_species │
╞════╪═════════╪═══════╪═══════╪═══════════╪═══════════╪════════════════╪══════════╪═════════════════╪════════════╪═════════════╪═══════════╪═════════════╡
│  0 │       3 │    40 │     0 │         2 │         9 │              3 │       11 │               4 │          1 │           4 │        12 │          24 │
╘════╧═════════╧═══════╧═══════╧═══════════╧═══════════╧════════════════╧══════════╧═════════════════╧════════════╧═════════════╧═══════════╧═════════════╛


/Users/thearsophal/opt/miniconda3/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/thearsophal/opt/miniconda3/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Amoxicilline : class -  0 Resistance  - Probab: 0.185
Augmentin : class -  0 Resistance  - Probab: 0.315
Oxacilline / cefazoline : class -  0 Resistance  - Probab: 0.025
Tazocilline : class -  1 Sensible  - Probab: 0.505
Cefotaxime / ceftriaxone : class -  0 Resistance  - Probab: 0.275
Ceftazidime : class -  0 Resistance  - Probab: 0.23
Cefepime : class -  0 Resistance  - Probab: 0.295
Aztreonam : class -  0 Resistance  - Probab: 0.17
Imipenem : class -  1 Sensible  - Probab: 0.62
Meropenem : class -  1 Sensible  - Probab: 0.625
Ertapenem : class -  1 Sensible  - Probab: 0.54
Amikacine : class -  0 Resistance  - Probab: 0.37
Gentamicine : class -  0 Resistance  - Probab: 0.25
Ciprofloxacine : class -  0 Resistance  - Probab: 0.145
Levofloxacine : class -  0 Resistance  - Probab: 0.14
Bactrim : class -  0 Resistance  - Probab: 0.205
Vancomycine : class -  0 Resistance  - Probab: 0.16
Rifampicine : class -  0 Resistance  - Probab: 0.025
ClindamycineMacrolides : class -  0 Resistance  - P